In [29]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader, Dataset # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
from PIL import Image

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu or cpu

In [31]:
device

device(type='cuda')

In [42]:
from sklearn.model_selection import train_test_split
dataset = ImageFolder("skin/")
train_data, test_data, train_label, test_label = train_test_split(dataset.imgs, dataset.targets, test_size=0.15, random_state=42)

# ImageLoader Class

class ImageLoader(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = self.checkChannel(dataset) # some images are CMYK, Grayscale, check only RGB 
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, item):
        image = Image.open(self.dataset[item][0])
        classCategory = self.dataset[item][1]
        if self.transform:
            image = self.transform(image)
        return image, classCategory
        
    
    def checkChannel(self, dataset):
        datasetRGB = []
        for index in range(len(dataset)):
            if (Image.open(dataset[index][0]).getbands() == ("R", "G", "B")): # Check Channels
                datasetRGB.append(dataset[index])
        return datasetRGB

In [52]:
train_transform = transforms.Compose([
    transforms.Resize((416, 416)), transforms.RandomHorizontalFlip(), transforms.ToTensor(),# transforms.Normalize([0.5]*3, [0.5]*3)
]) # train transform

test_transform = transforms.Compose([
    transforms.Resize((416, 416)), transforms.ToTensor(), # transforms.Normalize([0.5]*3, [0.5]*3)
]) # test transform

train_dataset = ImageLoader(train_data, train_transform)
test_dataset = ImageLoader(test_data, test_transform)

In [53]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [185]:
from tqdm import tqdm
from torchvision import models
from collections import OrderedDict
from timm.models.layers.activations import *
import timm
# load pretrain model and modify...
# model = models.EfficientNet(pretrained=True)
model = timm.create_model('efficientnet_b0', pretrained=True, drop_rate=0.2)

# If you want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.

for param in model.parameters():
    param.requires_grad = True

# # num_ftrs = model.fc.in_features
# fc = nn.Sequential(OrderedDict([
# 								#  ('fc1', nn.Linear(2048, 1000, bias=True)),
# 							    #  ('BN1', nn.BatchNorm2d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
# 								#  ('dropout1', nn.Dropout(0.7)),
#                                  ('fc2', nn.Linear(1280, 512)),
# 								 ('BN2', nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
# 								 ('swish1', Swish()),
# 								 ('dropout2', nn.Dropout(0.5)),
# 								 ('fc3', nn.Linear(512, 128)),
# 								 ('BN3', nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
# 							     ('swish2', Swish()),
# 								 ('fc4', nn.Linear(128, 3)),
# 								 ('output', nn.Softmax(dim=1))
# 							 ]))


# model.classifier = nn.Sequential(model.classifier, nn.Linear(1000, 512), nn.Linear(512, 3), nn.Softmax(dim=1))
# for param in model.classifier.parameters():
# 	param.requires_grad = True

model.to(device)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [ ]:
from tqdm import tqdm
from torchvision import models
from collections import OrderedDict
from timm.models.layers.activations import *
import timm
# load pretrain model and modify...
# model = models.EfficientNet(pretrained=True)
model = models.resnet34(pretrained=True)

# If you want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.

for param in model.parameters():
    param.requires_grad = True

num_ftrs = model.fc.in_features
fc = nn.Sequential(OrderedDict([#('fc1', nn.Linear(2048, 1000, bias=True)),
# 							     ('BN1', nn.BatchNorm2d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
# 								 ('dropout1', nn.Dropout(0.7)),
#                                  ('fc2', nn.Linear(1000, 512)),
								#  ('BN2', nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
								#  ('swish1', Swish()),
								#  ('dropout2', nn.Dropout(0.5)),
								#  ('fc3', nn.Linear(num_ftrs, 128)),
								#  ('BN3', nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
							    #  ('swish2', Swish()),
								 ('fc4', nn.Linear(num_ftrs, 3)),
								 ('output', nn.Softmax(dim=1))
							 ]))


model.fc = fc

model.to(device)

In [186]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer.zero_grad()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [187]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            _, predictions = torch.max(output, 1)
            correct += (predictions == y).sum().item()
            test_loss = criterion(output, y)
            
    test_loss /= len(test_loader.dataset)
    print("Average Loss: ", test_loss, "  Accuracy: ", correct, " / ",
    len(test_loader.dataset), "  ", int(correct / len(test_loader.dataset) * 100), "%")
    return correct / len(test_loader.dataset)

In [188]:
# Train and test

def train(num_epoch, model, patience = 10):
    best_acc = 0
    patience_count = 0
    
    for epoch in range(0, num_epoch):
#         current_loss = 0.0
#         current_corrects = 0
        losses = []
        model.train()
        loop = tqdm(enumerate(train_loader), total=len(train_loader)) # create a progress bar
        for batch_idx, (data, targets) in loop:
            if len(data) != 4:
                break
            data = data.to(device=device)
            targets = targets.to(device=device)
            scores = model(data)
            
            loss = criterion(scores, targets)
            optimizer.zero_grad()
            losses.append(loss)
            loss.backward()
            optimizer.step()
            _, preds = torch.max(scores, 1)
#             current_loss += loss.item() * data.size(0)
#             current_corrects += (preds == targets).sum().item()
#             accuracy = int(current_corrects / len(train_loader.dataset) * 100)
            loop.set_description(f"Epoch {epoch+1}/{num_epoch} process: {int((batch_idx / len(train_loader)) * 100)}")
            loop.set_postfix(loss=loss.data.item())
        
        test_acc = test()
        if test_acc > best_acc:
            best_acc = test_acc
            print('NEW BEST=', best_acc)
            patience_count = 0
            # save model
            torch.save({ 
                        'model_state_dict': model.state_dict(), 
                        'optimizer_state_dict': optimizer.state_dict(), 
                        }, 'bestmodel.pt')
        else:
            patience_count += 1
            if patience_count == patience:
                print('EARLY STOPPING, BEST ACC=', best_acc)
                break
        scheduler.step()

In [189]:
train(200, model) # train

Epoch 1/200 process: 99: 100%|█████████▉| 212/213 [00:14<00:00, 15.11it/s, loss=0.537]


Average Loss:  tensor(0.0025, device='cuda:0')   Accuracy:  103  /  150    68 %
NEW BEST= 0.6866666666666666


Epoch 2/200 process: 99: 100%|█████████▉| 212/213 [00:13<00:00, 15.26it/s, loss=0.781]


Average Loss:  tensor(0.0085, device='cuda:0')   Accuracy:  80  /  150    53 %


Epoch 3/200 process: 99: 100%|█████████▉| 212/213 [00:13<00:00, 15.15it/s, loss=1.38] 


Average Loss:  tensor(0.0111, device='cuda:0')   Accuracy:  115  /  150    76 %
NEW BEST= 0.7666666666666667


Epoch 4/200 process: 99: 100%|█████████▉| 212/213 [00:13<00:00, 15.23it/s, loss=0.294]


Average Loss:  tensor(0.0093, device='cuda:0')   Accuracy:  115  /  150    76 %


Epoch 5/200 process: 21:  22%|██▏       | 46/213 [00:03<00:10, 15.26it/s, loss=0.403]


KeyboardInterrupt: 

## Evaludate

In [120]:
test()

Average Loss:  tensor(0.0004, device='cuda:0')   Accuracy:  117  /  150    78 %


0.78

## Inference

restnet_50 0.645  
efficient_b0 78.0

In [192]:
from tqdm import tqdm
from torchvision import models
from collections import OrderedDict
from timm.models.layers.activations import *
import timm
# load pretrain model and modify...
# model = models.EfficientNet(pretrained=True)
model = timm.create_model('efficientnet_b0', pretrained=False, drop_rate=0.2)
# num_ftrs = model.fc.in_features
fc = nn.Sequential(OrderedDict([('fc1', nn.Linear(2048, 1000, bias=True)),
							     ('BN1', nn.BatchNorm2d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
								 ('dropout1', nn.Dropout(0.7)),
                                 ('fc2', nn.Linear(1000, 512)),
								 ('BN2', nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
								 ('swish1', Swish()),
								 ('dropout2', nn.Dropout(0.5)),
								 ('fc3', nn.Linear(512, 128)),
								 ('BN3', nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
							     ('swish2', Swish()),
								 ('fc4', nn.Linear(128, 3)),
								 ('output', nn.Softmax(dim=1))
							 ]))

model.fc = fc


In [193]:
print("----> Loading checkpoint")
checkpoint = torch.load("./efficient_b0_78.pt") # Try to load last checkpoint
model.load_state_dict(checkpoint["model_state_dict"]) 
model.to(device)
model.eval()
print('loaded')

----> Loading checkpoint
loaded


In [212]:
def RandomImagePrediction(filepath):
    img_array = Image.open(filepath).convert("RGB")
    data_transforms=transforms.Compose([
        transforms.Resize((416, 416)), 
        transforms.ToTensor(), 
        # transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    img = data_transforms(img_array).unsqueeze(dim=0) # Returns a new tensor with a dimension of size one inserted at the specified position.
    load = DataLoader(img)
    
    for x in load:
        x=x.to(device)
        pred = model(x)[0][:3].cpu().detach().numpy()
        return {
            'Level O':pred[0], 
            'Level 1': pred[1], 
            'Level 2': pred[2], 
        }

In [213]:
RandomImagePrediction("skin/Level_1/levle1_133.jpg")

{'Level O': 5.2535825, 'Level 1': 12.466472, 'Level 2': 3.342614}